# AngularJS feat. requireJS (as dependency to Jupyter Notebook)

In [1]:
#standard IPython modules for working with frontend output
from IPython.display import SVG, Javascript,HTML,display
#Gotta pimp the package that makes things easier
import simplejson as json

## A basic partial template file with AngularJS directives

In [2]:
#We can just read the file with Jupyter's shell commands "!"
!cat templates/input_template.ng

cat: templates/input_template.ng: No such file or directory


## We can load the dependencies files by reading into IPython.display.javascript

In [3]:
#Dependency js, although it can't be loaded via Javascript() the file for display
!cat js/deps.js

cat: js/deps.js: No such file or directory


## Prepare a python dictionary with some values
`kernel.execute("print(python_dict)")` will be executed by javascript

In [4]:
python_dict = {"input":"templates/input_template.ng","input_id":"content"}
json.dumps(python_dict)

'{"input": "templates/input_template.ng", "input_id": "content"}'

In [5]:
%%html
<H2>We just create a container to hold the content of our angular template
<br> 
(%%html is magic to define html to be rendered)</h2>
<H3>It'll be empty until we run the below...or until the autosave runs</h3>
<code>Note the html ID="container"</code>
<hr/>
<div id="container">Not Loaded...</div>
<hr/>

### Now with some %%javascript magic, read the notes as we step through the levels

In [6]:
%%javascript
//require needs to be configed with the dependencies
require.config({
    paths: {
        velocity: "https://cdn.jsdelivr.net/velocity/1.2.3/velocity.min",
        interact: "https://cdn.jsdelivr.net/interact.js/1.2.6/interact.min",
        angular: "https://cdnjs.cloudflare.com/ajax/libs/angular.js/1.5.8/angular.min"
       },
    shim: {
        'angular': {
            exports: 'angular'
        }
    }
});
//top level function will be fed into our callback
function handle_output(data){
    //data is the object passed to the callback from the kernel execution
    console.log(data)
    //to get the print statement from the text, we get the content key
    //then we convert it via JSON parse
    //now you can arbitrarily transfer a JSON string using the python_dict variable
    var python_dict = JSON.parse(data.content.text.trim())
    console.log(python_dict)
    //Utilized requireJS built into the frontend of Jupyter Notebook
    //Pass in the modular dependencies you wish to use
    //For this example, we created our own and passed it into the config obj
    // paths: {
    //    velocity: "https://cdn.jsdelivr.net/velocity/1.2.3/velocity.min",
    //    interact: "https://cdn.jsdelivr.net/interact.js/1.2.6/interact.min",
    //    angular: "https://cdnjs.cloudflare.com/ajax/libs/angular.js/1.5.8/angular.min"
    //   }
    //jquery is already available
    //find all the modules here
    console.log(require.s.contexts._.defined)
    //now we get to the requirejs -> angular
    require(['angular','jquery'], function(a,$) {
        console.log(a)
        //a and $ are now our variables for the modules
        //we using an ajax call to the jupyter notebook server
        //here we get the variable passed in, and us the input to locate the file
        $.ajax({url: "http://localhost:8888/files/"+python_dict.input,
            //once the file is retrieved, we have a callback, so we're one step deeper
            success: function(result) {
                //now we locate the ID="contaner" which is defined above %%html magic
                //pass out file result
                console.log('SUCCESS')
                $("#container").html(result);
                //now select the element we just loaded as the result (we pull from the python_dict)
                var el = document.getElementById(python_dict.input_id);
                //We have to be prepared to teardown and restart our controller
                //if the injector() is available, then angular as been bootstrapped
                //so destroy the scope and tree to reboot.
                if(a.element(el).injector()){
                    a.element(el).injector().get('$rootScope').$destroy()
                }
                //Now we get to the actual module, the a here was conntected to the angular module
                //myApp is only connected to the bootstrap
                //at this point, all angular stuff is fair game, and you can define your modules and controllers
                //nothing special for this hello world.
                a.module('myApp', [])
                  .controller('MyController', ['$scope', function ($scope) {
                    $scope.greeting = 'Hello';
                    $scope.name = 'Darling!!!';
                  }]);
                //we bootstrap ("INITIALIZE") the application so that we can easily keep control
                angular.element(document).ready(function() {
                  angular.bootstrap(el, ['myApp']);
                });
            }
        });
    });
    
}
//callbacks is an object whose so special, it appears to only have been documented in
//the source code, as no only google found me a link.
//callbacks.iopub.output is used to get the data from execute
var callbacks = {
        iopub : {
             output : handle_output,
    }
}
//execute anything you want; if a string value is returned
//you can print it out and pass it to the callbacks 
//(or do other things, no idea, it's poorly documented online 
//(read the source F12->static/notebook/js/services/kernels/kernel.js)
//kernel.js/Kernel.prototype.execute 
var kernel = IPython.notebook.kernel;
kernel.execute("print(json.dumps(python_dict))",callbacks)

<IPython.core.display.Javascript object>